In [1]:
import kglab
import os

if 'src' in os.getcwd():
    os.chdir('..')

os.getcwd()

'/mnt/c/repos/rdf_permissions_example'

In [2]:
g = kglab.KnowledgeGraph()
g.load_rdf("data/server/graph_data.rdf", format="xml")

In [3]:
# Since we haven't run ontological reasoning yet, the "manages" relationships are not inferred
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = g.query_as_df(query)
res

,s,p,o
0,<local://security.permissions.net/entities#Anya>,:managesDepartment,<local://security.permissions.net/entities#STO...
1,<local://security.permissions.net/entities#Jac...,:managesDepartment,<local://security.permissions.net/entities#APP...
2,<local://security.permissions.net/entities#John>,:managesDepartment,<local://security.permissions.net/entities#OFF...
3,<local://security.permissions.net/entities#Mary>,:managesDepartment,<local://security.permissions.net/entities#BIR...
4,<local://security.permissions.net/entities#Sam...,:managesDepartment,<local://security.permissions.net/entities#HOM...
5,<local://security.permissions.net/entities#John>,:managesDepartment,<local://security.permissions.net/entities#HOBBY>
6,<local://security.permissions.net/entities#Steve>,:managesDepartment,<local://security.permissions.net/entities#KIT...
7,<local://security.permissions.net/entities#Mary>,:managesDepartment,<local://security.permissions.net/entities#CHR...


In [4]:
# Run the rules engine - takes a crazy long amount of time
g.infer_owlrl_closure()

In [7]:
# Now that we have added our inferences, we can see the results of the manages relationships which have been inferred from both managesDepartment and mangedBy
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = g.query_as_df(query)
res

,s,p,o
0,<local://security.permissions.net/entities#Mary>,:manages,<local://security.permissions.net/entities#Sal...
1,<local://security.permissions.net/entities#Mary>,:manages,<local://security.permissions.net/entities#Sal...
2,<local://security.permissions.net/entities#Jac...,:managesDepartment,<local://security.permissions.net/entities#APP...
3,<local://security.permissions.net/entities#John>,:manages,<local://security.permissions.net/entities#Sal...
4,<local://security.permissions.net/entities#Jac...,:manages,<local://security.permissions.net/entities#Sal...
...,...,...,...
61,<local://security.permissions.net/entities#John>,:manages,<local://security.permissions.net/entities#Sal...
62,<local://security.permissions.net/entities#Sam...,:manages,<local://security.permissions.net/entities#Sal...
63,<local://security.permissions.net/entities#Jac...,:manages,<local://security.permissions.net/entities#Sal...
64,<local://security.permissions.net/entities#Mary>,:managesDepartment,<local://security.permissions.net/entities#CHR...


In [ ]:
# Find all the records that a specific user may have access to.
# Takes a very very long time

USER = "SalesRep_0"

query = f"""
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select ?record where {{
?department a :Department .
?department :hasItem ?item .
{{
data:{USER} :hasAccess ?country, ?department ; a :User .
?country a :Country .
?record a :Record; :country ?country; :item ?item.}}
union {{
data:{USER} a :Manager .
data:{USER} :managesDepartment ?department .
?record a :Record; :item ?item.
}}
}}
"""

# Execute the query
res = g.query_as_df(query)
res

KeyboardInterrupt: 